In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ilsilfverskiold/tech-keywords-extractor")
model = AutoModelForSeq2SeqLM.from_pretrained("ilsilfverskiold/tech-keywords-extractor")

c:\Users\carlv\eth\24_HS\semester project\technology_complexity\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This model finds the most important keywords in a sentence. This is used to filter the claims text to an accaptable length and then calculate the entropy.

In [6]:
sequence = "I have a problem with my code. I am trying to run a python script that uses the pandas library. However, I keep getting an error message that says 'no module named pandas'. I have tried reinstalling the library, but that did not fix the issue. Can someone help me troubleshoot this problem?"

tokens = tokenizer(sequence, padding=True, truncation=True, return_tensors="pt")

summary = model.generate(**tokens)

In [8]:
tokenizer.decode(summary[0], skip_special_tokens=True)

'Python, Pandas, Error Message'